In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
!python -m nltk.downloader punkt

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!git clone https://github.com/ZZZZkp/question_generation.git

Cloning into 'question_generation'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 210 (delta 13), reused 56 (delta 10), pack-reused 148
Receiving objects: 100% (210/210), 76.86 MiB | 23.64 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [3]:
%cd question_generation

/content/question_generation


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [5]:
from tqdm.auto import tqdm
import logging

In [6]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, HfArgumentParser

In [7]:
from data_collator import T2TDataCollator

In [8]:
from pipelines import pipeline

##Get a pretrained t5 model.

In [9]:
nlp = pipeline("question-generation", model="valhalla/t5-base-qg-hl", ans_model="valhalla/t5-base-qa-qg-hl", use_cuda=False)

tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

## Try to give it a sentence, it will generate a question.

In [10]:
nlp("42 is the answer to life, the universe and everything.")

[{'answer': '42',
  'question': 'What is the answer to life, the universe and everything?'}]

## Preprocess the data of SQuAD and SQuAD NQG


In [13]:
!python prepare_data.py \
    --task qg \
    --model_type t5 \
    --dataset_path data/squad_multitask/ \
    --qg_format highlight_qg_format \
    --max_source_length 512 \
    --max_target_length 32 \
    --train_file_name train_data_squad_t5.pt \
    --valid_file_name valid_data_squad_t5.pt

spiece.model: 100% 792k/792k [00:00<00:00, 8.98MB/s]
tokenizer.json: 100% 1.39M/1.39M [00:00<00:00, 3.35MB/s]
config.json: 100% 1.21k/1.21k [00:00<00:00, 7.55MB/s]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:926: FutureWarning: The repository for squad_multitask contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at data/squad_multitask/squad_

In [14]:
!python prepare_data_squad_nqg.py --task qg --model_type t5 --dataset_path data/squad_nqg/ --qg_format highlight_qg_format \
    --max_source_length 512 \
    --max_target_length 32 \
    --train_file_name train_data_squad_nqg_t5.pt \
    --valid_file_name test_data_squad_nqg_t5.pt \
    --create_reference true \
    --reference_file_name squad_nqg_tgt.txt


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:926: FutureWarning: The repository for squad_nqg contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at data/squad_nqg/squad_nqg.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the n

## Use the test data to inference the result
(These data is already inferenced, you can skip this step)

In [15]:
!python eval.py \
    --model_name_or_path valhalla/t5-base-qg-hl \
    --valid_file_path data/valid_data_squad_t5.pt \
    --model_type t5 \
    --num_beams 4 \
    --max_decoding_length 32 \
    --output_path hypothesis_squad.txt

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100% 331/331 [06:54<00:00,  1.25s/it]


In [16]:
!python eval.py \
    --model_name_or_path valhalla/t5-base-qg-hl \
    --valid_file_path data/test_data_squad_nqg_t5.pt \
    --model_type t5 \
    --num_beams 4 \
    --max_decoding_length 32 \
    --output_path hypothesis_squad_nqg.txt

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100% 372/372 [07:36<00:00,  1.23s/it]


## Evaluate the result of two datasets over the following metrics: BLEU-1,2, 3, 4, METEOR, ROUGE-L.

In [17]:
!python qgevalcap/eval.py --out_file hypothesis_squad.txt --src_file data/src-dev.txt --tgt_file data/references.txt

scores: 

Bleu_1: 0.49384
Bleu_2: 0.37385
Bleu_3: 0.29416
Bleu_4: 0.23658
METEOR: 0.28131
ROUGE_L: 0.45438


In [18]:
!python qgevalcap/eval.py --out_file hypothesis_squad_nqg.txt --src_file data/src-test.txt --tgt_file data/squad_nqg_tgt.txt

scores: 

Bleu_1: 0.52012
Bleu_2: 0.40475
Bleu_3: 0.32545
Bleu_4: 0.26579
METEOR: 0.29291
ROUGE_L: 0.47647
